# Plug flow reactor simulation of Thruster

![caption](Graphics/thruster-details.png)


In [1]:
import cantera as ct
import numpy as np

from matplotlib import pyplot as plt
import csv
import os
import itertools
import pandas as pd

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
import json
with open('rocketman/settings.json') as fp:
    settings = json.load(fp)
cat_area_per_vol_options = list(sorted(set(np.array(settings)[:,0])))
temperature_c_options = list(sorted(set(np.array(settings)[:,1])))
rtol_options= list(sorted(set(np.array(settings)[:,2])))
atol_options= list(sorted(set(np.array(settings)[:,3])))
        

In [4]:
cat_area_per_vol_options, temperature_c_options, rtol_options, atol_options

([3000.0, 30000.0, 300000.0, 3000000.0, 30000000.0, 300000000.0],
 [200.0, 300.0, 400.0, 500.0],
 [1e-11, 1e-09, 1e-06],
 [1e-24, 1e-21, 1e-18, 1e-15])

In [5]:
f'1e{np.log10(100):g}', f'{1000:.0e}'

('1e2', '1e+03')

In [6]:

print("Processing SLURM array job.")

rocket_array_dir = 'rocketman'

cti_file = os.path.join(rocket_array_dir,'0','chem_annotated.cti')

print(f"Using cantera input file {os.path.abspath(cti_file)}")

print(f"Settings array is from 0 to {len(settings)-1} ")

from collections import defaultdict
setting_dict = dict()
for i,(area,temp,rtol,atol) in enumerate(settings):
    setting_dict[(area,temp,rtol,atol)] = i


Processing SLURM array job.
Using cantera input file /Users/rwest/Dropbox (CoMoChEng)/Northeastern/Research-MHI/HAN (github)/CanteraModels/rocketman/0/chem_annotated.cti
Settings array is from 0 to 287 


In [7]:
data_dict = dict()
for i,(area,temp,rtol,atol) in enumerate(settings):
    output_filename = os.path.join('rocketman',str(i),'surf_pfr_output.csv')
    print(f"{i:3d} {area:.0e}/m {temp:.0f}K rt{rtol:.0e} at{atol:.0e}", end=' ')
    try:
        data = pd.read_csv(output_filename)
        print(f"✅ OK  {data['Distance (mm)'].max():5.2f} mm    {data['T (C)'].min():.0f}-{data['T (C)'].max():.0f} ºC")
    except:
        print("❌ FAIL!")
        data = None
    data_dict[(area,temp,rtol,atol)] = data

  0 3e+03/m 200K rt1e-06 at1e-15 ✅ OK  11.00 mm    197-200 ºC
  1 3e+03/m 200K rt1e-06 at1e-18 ✅ OK  11.00 mm    197-200 ºC
  2 3e+03/m 200K rt1e-06 at1e-21 ✅ OK  11.00 mm    197-200 ºC
  3 3e+03/m 200K rt1e-06 at1e-24 ✅ OK  11.00 mm    197-200 ºC
  4 3e+03/m 200K rt1e-09 at1e-15 ✅ OK  11.00 mm    197-200 ºC
  5 3e+03/m 200K rt1e-09 at1e-18 ✅ OK  11.00 mm    197-200 ºC
  6 3e+03/m 200K rt1e-09 at1e-21 ✅ OK  11.00 mm    197-200 ºC
  7 3e+03/m 200K rt1e-09 at1e-24 ✅ OK  11.00 mm    197-200 ºC
  8 3e+03/m 200K rt1e-11 at1e-15 ✅ OK  11.00 mm    197-200 ºC
  9 3e+03/m 200K rt1e-11 at1e-18 ✅ OK  11.00 mm    198-200 ºC
 10 3e+03/m 200K rt1e-11 at1e-21 ✅ OK  11.00 mm    198-200 ºC
 11 3e+03/m 200K rt1e-11 at1e-24 ✅ OK  11.00 mm    198-200 ºC
 12 3e+03/m 300K rt1e-06 at1e-15 ✅ OK  11.00 mm    299-300 ºC
 13 3e+03/m 300K rt1e-06 at1e-18 ✅ OK  11.00 mm    299-300 ºC
 14 3e+03/m 300K rt1e-06 at1e-21 ✅ OK  11.00 mm    299-300 ºC
 15 3e+03/m 300K rt1e-06 at1e-24 ✅ OK  11.00 mm    298-300 ºC
 16 3e+0

134 3e+05/m 500K rt1e-06 at1e-21 ✅ OK  11.00 mm    495-500 ºC
135 3e+05/m 500K rt1e-06 at1e-24 ✅ OK  11.00 mm    495-503 ºC
136 3e+05/m 500K rt1e-09 at1e-15 ✅ OK  11.00 mm    496-500 ºC
137 3e+05/m 500K rt1e-09 at1e-18 ✅ OK  11.00 mm    495-500 ºC
138 3e+05/m 500K rt1e-09 at1e-21 ✅ OK  11.00 mm    495-500 ºC
139 3e+05/m 500K rt1e-09 at1e-24 ✅ OK  11.00 mm    500-500 ºC
140 3e+05/m 500K rt1e-11 at1e-15 ✅ OK  11.00 mm    495-500 ºC
141 3e+05/m 500K rt1e-11 at1e-18 ✅ OK  11.00 mm    495-500 ºC
142 3e+05/m 500K rt1e-11 at1e-21 ✅ OK  11.00 mm    496-500 ºC
143 3e+05/m 500K rt1e-11 at1e-24 ✅ OK  11.00 mm    496-500 ºC
144 3e+06/m 200K rt1e-06 at1e-15 ✅ OK   1.09 mm    -152-200 ºC
145 3e+06/m 200K rt1e-06 at1e-18 ✅ OK   1.73 mm    -97-200 ºC
146 3e+06/m 200K rt1e-06 at1e-21 ✅ OK   3.24 mm    -135-200 ºC
147 3e+06/m 200K rt1e-06 at1e-24 ✅ OK   5.74 mm    -142-200 ºC
148 3e+06/m 200K rt1e-09 at1e-15 ✅ OK  11.00 mm    -148-200 ºC
149 3e+06/m 200K rt1e-09 at1e-18 ✅ OK  11.00 mm    -137-200 ºC
150

270 3e+08/m 400K rt1e-09 at1e-21 ✅ OK  11.00 mm    -17-400 ºC
271 3e+08/m 400K rt1e-09 at1e-24 ✅ OK  11.00 mm    5-400 ºC
272 3e+08/m 400K rt1e-11 at1e-15 ✅ OK  11.00 mm    -0-400 ºC
273 3e+08/m 400K rt1e-11 at1e-18 ✅ OK  11.00 mm    209-400 ºC
274 3e+08/m 400K rt1e-11 at1e-21 ✅ OK  11.00 mm    253-400 ºC
275 3e+08/m 400K rt1e-11 at1e-24 ✅ OK  11.00 mm    339-400 ºC
276 3e+08/m 500K rt1e-06 at1e-15 ✅ OK   1.52 mm    -103-500 ºC
277 3e+08/m 500K rt1e-06 at1e-18 ✅ OK   3.36 mm    -105-500 ºC
278 3e+08/m 500K rt1e-06 at1e-21 ✅ OK   7.26 mm    -118-500 ºC
279 3e+08/m 500K rt1e-06 at1e-24 ✅ OK   9.72 mm    -130-500 ºC
280 3e+08/m 500K rt1e-09 at1e-15 ✅ OK   5.78 mm    -111-500 ºC
281 3e+08/m 500K rt1e-09 at1e-18 ✅ OK  11.00 mm    -107-500 ºC
282 3e+08/m 500K rt1e-09 at1e-21 ✅ OK  11.00 mm    19-500 ºC
283 3e+08/m 500K rt1e-09 at1e-24 ✅ OK  11.00 mm    500-500 ºC
284 3e+08/m 500K rt1e-11 at1e-15 ✅ OK  11.00 mm    191-500 ºC
285 3e+08/m 500K rt1e-11 at1e-18 ✅ OK  11.00 mm    385-500 ºC
286 3e

In [8]:
gas=ct.Solution(cti_file)
surf = ct.Interface(cti_file,'surface1', [gas])

In [9]:
print(", ".join(gas.species_names))

Ne, NH3(2), NH2OH(3), HNO3(4), CH3OH(5), H2O(6), N2(7), O2(8), NO2(9), NO(10), N2O(11), CO2(12), H2(13), CO(14), CH4(15), C2H6(16), CH2O(17), CH3(18), C3H8(19), H(20), C2H5(21), HCO(22), CH3CHO(23), OH(24), C2H4(25), O(36), Ar(37), HO2(39), H2O2(40), HOCO(41), CH2(42), CH2(S)(43), CH(44), CH2OH(45), CH3O(46), HCOH(47), CH3OO(48), CH2CO(49), C2H3(50), C(51), C2H2(52), C2H(53), CH3OOH(54), CH2OOH(55), HOCH2O(56), HOCHO(57), C2H5O(58), C2H5O2(59), C2H5O2(60), cC2H4O(61), CH2CHO(62), H2CC(63), CH3CO(64), C2H4O(65), C2H5O(66), C2H3O2(67), CHCHO(68), OCHCHO(69), HCCO(70), HCCOH(71), CHCHOH(72), C2(73), C2O(74), C2H6O(75), C2H5O(76), C2H5O3(77), cC2H3O(78), C2H3O3(79), OCHCO(80), C2H6O2(81), C2H5O2(82), C2H4O2(83), OCHO(84), NH2(85), NH(86), HNO(87), H2NO(88), HON(89), N(90), NNH(91), HONO(92), HNOH(93), HNO2(94), NO3(95), N2H2(96), H2N2(97), N2H3(98), N2H4(99), HCN(100), CN(101), HNC(102), NCO(103), HOCN(104), HNCO(105), NCCN(106), HNCN(107), NCN(108), HNCNH(109), HCNO(110), CH3CN(111), CH2C

In [10]:
print(", ".join(surf.species_names))

X(1), HX(26), OX(27), CH3X(28), HOX(29), H2OX(30), CO2X(31), OCX(32), CX(33), CH2X(34), CHX(35), H2NX(201), H3NX(202), H2NOX(203), H2NOX(204), H3NOX(205), HNO3X(208), CH3OX(210), CH4OX(211), NO2X(214), NOX(215), NOJX(216), H2X(218), CH2OX(224), HNX(519), HNOX(520), CH2OX(522), H2NOX(533), CHOX(588), SX(589), SX(590), HNO2X(594), NX(617), SX(618), CH4NX(666), CH3NX(667), CH2NX(668), H2N2X(671), HONOX(672), HNOX(744), SX(935), CNOX(943), N2OX(946), SX(955), SX(1252), SX(1484)


In [11]:
# unit conversion factors to SI
cm = 0.01 # m
minute = 60.0  # s

In [12]:
#######################################################################
# Input Parameters for combustor
#######################################################################
mass_flow_rate =  0.5e-3 # kg/s

pressure = ct.one_atm # constant

length = 1.1 * cm  # Catalyst bed length. 11mm
cross_section_area = np.pi * (0.9*cm)**2  # Catalyst bed area.  18mm diameter circle.


In [13]:
NReactors = 2001
def xlabels():
    ticks = []
    labels = []
    mm = 0
    while mm < length*1000:
        ticks.append( int(NReactors * mm * 0.001 / length ) )
        labels.append( str(mm) )
        mm += 1
    labels[-1] = labels[-1] + ' mm'
    plt.xticks(ticks, labels)
    plt.xlabel("Distance down reactor")

In [14]:
data = next(iter(data_dict.values()))
specs = list(data.columns)
specs = specs[4:-3]
excluded = [s for s in data.columns if s not in specs]
gas_species = [s for s in specs if 'X' not in s ]
adsorbates = [s for s in specs if 'X' in s]

In [15]:
def f(cat_area_per_vol, temperature_c, rtol, atol):
    
    print(f"Catalyst area per volume {cat_area_per_vol :.2e} m2/m3")
    print(f"Initial temperature {temperature_c :.1f} ºC")
    print(f"Solver RTol {rtol :.1e}")
    print(f"Solver ATol {atol :.1e}")
    setting_tuple = (cat_area_per_vol, temperature_c, rtol, atol)
    print(f"Simulation number {setting_dict[setting_tuple]}")
    data = data_dict[setting_tuple]
    
    data['T (C)'].plot()
    plt.ylabel('T (C)')
    xlabels()
    plt.show()
    
    data[['gas_heat','surface_heat']].plot()
    xlabels()
    #plt.savefig('gas_and_surface_heat.pdf')
    plt.show()

    species_to_plot = ['NH3(2)',
 'NH2OH(3)',
 'HNO3(4)',
 'CH3OH(5)',
 'H2O(6)',
 'N2(7)',
 'O2(8)',
 'NO2(9)',
 'NO(10)',
 'N2O(11)',
        ]
    data[species_to_plot].plot(title='gas mole fraction', logy=False)
    xlabels()
    plt.tight_layout()
    #plt.savefig(f'gas_mole_fractions_{i}.pdf')
    plt.show()
    
    main_adsorbates = data[adsorbates].max().sort_values(ascending=False)[:10].keys()
    data[main_adsorbates].plot.area()
    xlabels()
    plt.xlim(0,len(data)+5)
    plt.tight_layout()
    plt.savefig(f'surface_coverages_top10.pdf')
    plt.show()

    return data
    

area_ = widgets.SelectionSlider(
    options=cat_area_per_vol_options,
    value=3e5,
    description='Catalyst area per volume (m2/m3)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
temp_ = widgets.SelectionSlider(
    options=temperature_c_options,
    value=400,
    description='Initial temperature (C)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

rtol_ = widgets.SelectionSlider(
    options=rtol_options,
    value=rtol_options[1],
    description='RTol',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

atol_ = widgets.SelectionSlider(
    options=atol_options,
    value=atol_options[1],
    description='ATol',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)



interact(f, cat_area_per_vol=area_, temperature_c=temp_, rtol=rtol_, atol=atol_)


interactive(children=(SelectionSlider(continuous_update=False, description='Catalyst area per volume (m2/m3)',…

<function __main__.f(cat_area_per_vol, temperature_c, rtol, atol)>

In [16]:
data['Distance (mm)'].max()

11.000000000000002

In [17]:
def report_rates(n=8):
    print("\nHighest net rates of progress, gas")
    for i in np.argsort(abs(gas.net_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.net_rates_of_progress[i]:8.1g}")
    print("\nHighest net rates of progress, surface")
    for i in np.argsort(abs(surf.net_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.net_rates_of_progress[i]:8.1g}")
    print("\nHighest forward rates of progress, gas")
    for i in np.argsort(abs(gas.forward_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.forward_rates_of_progress[i]:8.1g}")
    print("\nHighest forward rates of progress, surface")
    for i in np.argsort(abs(surf.forward_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.forward_rates_of_progress[i]:8.1g}")
    print("\nHighest reverse rates of progress, gas")
    for i in np.argsort(abs(gas.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.reverse_rates_of_progress[i]:8.1g}")
    print("\nHighest reverse rates of progress, surface")
    for i in np.argsort(abs(surf.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.reverse_rates_of_progress[i]:8.1g}")

    print(f"\nSurface rates have been scaled by surface/volume ratio {cat_area_per_vol:.1e} m2/m3")
    print("So are on a similar basis of volume of reactor (though porosity not yet accounted for)")
    print(" kmol / m3 / s")
report_rates()


Highest net rates of progress, gas
1581 : C2H4N(160) + O(36) => CH3(18) + H(20) + NCO(103)         0
519 : C2H5(21) + O2(8) <=> OH(24) + cC2H4O(61)                 0
521 : C2H5(21) + CH3OO(48) <=> C2H5O(58) + CH3O(46)            0
522 : C2H3(50) + H(20) (+M) <=> C2H4(25) (+M)                  0
523 : C2H4(25) (+M) <=> H2(13) + H2CC(63) (+M)                 0
524 : C2H4(25) + H(20) <=> C2H3(50) + H2(13)                   0
525 : C2H4(25) + O(36) <=> CH3(18) + HCO(22)                   0

Highest net rates of progress, surface


NameError: name 'cat_area_per_vol' is not defined

In [ ]:
def report_rate_constants(n=8):
    print("\nHighest forward rate constants, gas")
    for i in np.argsort(abs(gas.forward_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.forward_rate_constants[i]:8.1e}")
    print("\nHighest forward rate constants, surface")
    for i in np.argsort(abs(surf.forward_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {surf.forward_rate_constants[i]:8.1e}")
    print("\nHighest reverse rate constants, gas")
    for i in np.argsort(abs(gas.reverse_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.reverse_rate_constants[i]:8.1e}")
    print("\nHighest reverse rate constants, surface")
    for i in np.argsort(abs(surf.reverse_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {surf.reverse_rate_constants[i]:8.1e}")

    print("Units are a combination of kmol, m^3 and s, that depend on the rate expression for the reaction.")
report_rate_constants()

In [ ]:
sim.time

In [ ]:
gas.TDY = TDY
r.syncState()
r.thermo.T

In [ ]:
r.thermo.X - gas.X

In [ ]:
report_rate_constants()

In [ ]:
def xlabels():
    plt.xticks([0,NReactors/4,NReactors/2,3*NReactors/4, NReactors],['0','','','',f'{length*1000:.0f} mm'])
    plt.xlabel("Distance down reactor")

In [ ]:
data['T (C)'].plot()
plt.ylabel('T (C)')
xlabels()

In [ ]:
data[['NH2OH(3)', 'HNO3(4)', 'CH3OH(5)']].plot()
plt.ylabel('Mole fraction')
xlabels()

In [ ]:
list(data.columns)[:4]

In [ ]:
data[['T (C)', 'alpha']].plot()
xlabels()

In [ ]:
ax1 = data['T (C)'].plot()
plt.ylabel('Temperature (C)')
xlabels()
plt.legend()
ax2 = ax1.twinx()
data['alpha'].plot(ax=ax2, color='tab:orange')
ax2.set_ylim(-2, 2)
plt.legend()
plt.ylabel('alpha')
plt.tight_layout()
plt.savefig('temperature-and-alpha.pdf')
plt.show()

In [ ]:
data.columns

In [ ]:
data[['gas_heat','surface_heat']].plot()
#plt.ylim(-1e7, 1e7)
xlabels()
plt.savefig('gas_and_surface_heat.pdf')
plt.show()


In [ ]:
ax1 = data[['gas_heat','surface_heat']].plot()
plt.ylim(-1e9, 1e9)
xlabels()
plt.ylabel('Heat consumption rate (kJ/m3/s)')
plt.legend(loc='upper left')
ax2 = ax1.twinx()
data['alpha'].plot(ax=ax2, style='k:', alpha=0.5)
ax2.set_ylim(-10, 10)
plt.legend(loc='lower right')
plt.ylabel('alpha')
plt.tight_layout()
plt.savefig('heats-and-alpha.pdf')
plt.show()

In [ ]:
data[['T (C)']].plot()
plt.ylabel('Temperature (C)')
xlabels()
plt.tight_layout()
plt.savefig('temperature.pdf')
plt.show()

In [ ]:
data[['alpha']].plot(logy=True)
xlabels()

In [ ]:
data.plot(x='T (C)',y='alpha')


In [ ]:
specs = list(data.columns)
specs = specs[4:-1]

gas_species = [s for s in specs if 'X' not in s]
adsorbates = [s for s in specs if 'X' in s]

gas_species, adsorbates

In [ ]:
data[gas_species[0:5]].plot(logy=True, logx=True)

In [ ]:
for i in range(0,len(gas_species),10):
    data[gas_species[i:i+10]].plot(title='gas mole fraction', logy=False)
    xlabels()
    plt.tight_layout()
    plt.savefig(f'gas_mole_fractions_{i}.pdf')
    plt.show()
    
for i in range(0,len(adsorbates),10):
    data[adsorbates[i:i+10]].plot(title='surface coverages', logy=False)
    xlabels()
    plt.tight_layout()
    plt.savefig(f'surface_coverages_{i}.pdf')
    plt.show()